### AIDI-1006 Assignment. done by Yash solanki (200579052) and Priyanka priyanka (200585516)

In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import sqlite3

In [18]:
df = pd.read_csv('sentimentdataset.csv')
df.sample(10)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Text,Sentiment,Timestamp,User,Platform,Hashtags,Retweets,Likes,Country,Year,Month,Day,Hour,cleaned_text
234,234,235,239,"Seeking serenity in the melody of raindrops, ...",Serenity,2020-04-12 19:30:00,RaindropHarmony,Twitter,#Serenity #RaindropMelody,25.0,50.0,Canada,2020,4,12,19,seeking serenity melody raindrop tranquil esca...
337,337,338,342,Overwhelmed by the support received during a p...,Overwhelmed,2021-05-20 17:30:00,PersonalJourney,Facebook,#Overwhelmed #SupportiveCommunity,25.0,50.0,USA,2021,5,20,17,overwhelmed support received personal challenge
229,229,230,234,Melancholy painting the world in hues of nost...,Melancholy,2020-01-05 20:00:00,CanvasDreamer,Instagram,#Melancholy #BittersweetMemories,13.0,26.0,USA,2020,1,5,20,melancholy painting world hue nostalgia canvas...
610,610,611,615,Embarked on a road trip to revisit cherished p...,Joy,2023-06-19 16:30:00,RoadTripSenior,Facebook,#CherishedPlaces #SeniorTravel,22.0,45.0,Canada,2023,6,19,16,embarked road trip revisit cherished place pas...
540,540,541,545,Celebrating a historic victory in the World Cu...,Joy,2018-07-15 21:30:00,FootballFanWorldCupCelebration,Instagram,#Joy #WorldCupTriumph,40.0,80.0,Brazil,2018,7,15,21,celebrating historic victory world cup nation ...
221,221,222,224,Flipping through the pages of an old yearbook...,Nostalgia,2017-06-18 14:45:00,YearbookExplorer,Instagram,#Nostalgia #YearbookMemories,12.0,25.0,India,2017,6,18,14,flipping page old yearbook nostalgia painting ...
226,226,227,231,"Indifferent to the noise of the world, a sile...",Indifference,2023-03-10 16:30:00,SilentSpectator,Instagram,#Indifference #SilentObserver,9.0,18.0,India,2023,3,10,16,indifferent noise world silent observer midst ...
451,451,452,456,"Haunted by the specter of lost possibilities, ...",Regret,2019-11-08 14:45:00,PossibilityGhost,Twitter,#Regret #FadingPossibilities,18.0,35.0,Spain,2019,11,8,14,haunted specter lost possibility ghost refuse ...
632,632,633,637,Hosted a photography exhibition featuring snap...,Gratitude,2023-07-11 18:45:00,SeniorPhotographerExhibitor,Twitter,#LifeInPictures #SeniorExhibition,22.0,45.0,Canada,2023,7,11,18,hosted photography exhibition featuring snapsh...
83,83,84,86,Amused by the antics of my pet—it's pure amus...,Amusement,2023-02-23 14:20:00,PetAmuser,Instagram,#Amusement #PetAntics,18.0,35.0,UK,2023,2,23,14,amused antic pet pure amusement


In [3]:

import re
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Load your dataset
df = pd.read_csv('sentimentdataset.csv')

# Apply the cleaning function to the 'Text' column
df['cleaned_text'] = df['Text'].apply(clean_text)

# Print the first few rows of the DataFrame to verify the results
print(df[['Text', 'cleaned_text']].head())



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yash\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Yash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yash\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                Text  \
0   Enjoying a beautiful day at the park!        ...   
1   Traffic was terrible this morning.           ...   
2   Just finished an amazing workout! 💪          ...   
3   Excited about the upcoming weekend getaway!  ...   
4   Trying out a new recipe for dinner tonight.  ...   

                       cleaned_text  
0       enjoying beautiful day park  
1          traffic terrible morning  
2          finished amazing workout  
3  excited upcoming weekend getaway  
4  trying new recipe dinner tonight  


In [4]:
# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the cleaned text data into TF-IDF features
X = vectorizer.fit_transform(df['cleaned_text']).toarray()

# Display the shape of the TF-IDF features
print("\nShape of TF-IDF Features:", X.shape)

# Display the first few rows of the TF-IDF features
print("\nFirst Few Rows of TF-IDF Features:")
print(pd.DataFrame(X, columns=vectorizer.get_feature_names_out()).head())



Shape of TF-IDF Features: (732, 2154)

First Few Rows of TF-IDF Features:
   ablaze  abstract  abyss  academic  acceptance  accepts  accidentally  \
0     0.0       0.0    0.0       0.0         0.0      0.0           0.0   
1     0.0       0.0    0.0       0.0         0.0      0.0           0.0   
2     0.0       0.0    0.0       0.0         0.0      0.0           0.0   
3     0.0       0.0    0.0       0.0         0.0      0.0           0.0   
4     0.0       0.0    0.0       0.0         0.0      0.0           0.0   

   accomplished  accomplishing  accomplishment  ...  year  yearbook  yearning  \
0           0.0            0.0             0.0  ...   0.0       0.0       0.0   
1           0.0            0.0             0.0  ...   0.0       0.0       0.0   
2           0.0            0.0             0.0  ...   0.0       0.0       0.0   
3           0.0            0.0             0.0  ...   0.0       0.0       0.0   
4           0.0            0.0             0.0  ...   0.0       0.0  

In [5]:
# Extract labels (sentiment) from the DataFrame
y = df['Sentiment']  # Assuming the sentiment labels are in the 'Sentiment' column

# Display the first few labels
print("\nFirst Few Labels:")
print(y.head())



First Few Labels:
0     Positive  
1     Negative  
2     Positive  
3     Positive  
4     Neutral   
Name: Sentiment, dtype: object


In [6]:
# Train-test split
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
# Display the shapes of the training and testing sets
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)


X_train shape: (585, 2154)
X_test shape: (147, 2154)
y_train shape: (585,)
y_test shape: (147,)


In [8]:
# Model training
from sklearn.linear_model import LogisticRegression
# Initialize the Logistic Regression model
model = LogisticRegression()
# Train the model using the training data
model.fit(X_train, y_train)
# Display the model's parameters
print("\nModel Parameters:")
print(model.get_params())




Model Parameters:
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'deprecated', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [9]:
from sklearn.metrics import classification_report, accuracy_score
import sqlite3


In [10]:
# Predict sentiment labels for the test set
y_pred = model.predict(X_test)


In [11]:
# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print the classification report
print(classification_report(y_test, y_pred))


Accuracy: 0.11564625850340136
                        precision    recall  f1-score   support

         Acceptance          0.00      0.00      0.00         2
           Admiration        0.00      0.00      0.00         1
        Admiration           0.00      0.00      0.00         1
         Affection           0.00      0.00      0.00         1
      Ambivalence            0.00      0.00      0.00         1
         Anger               0.00      0.00      0.00         1
        Anticipation         0.00      0.00      0.00         1
        Arousal              0.00      0.00      0.00         3
                  Awe        0.00      0.00      0.00         1
         Awe                 0.00      0.00      0.00         1
                  Bad        0.00      0.00      0.00         1
             Betrayal        0.00      0.00      0.00         2
        Betrayal             0.00      0.00      0.00         1
         Bitter              0.00      0.00      0.00         1
         

C:\Users\Yash\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Yash\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Yash\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [12]:
# Save the cleaned dataset to a CSV file
df.to_csv('cleaned_sentiment_dataset.csv', index=False)


In [13]:
# Save the cleaned dataset to an SQLite database
conn = sqlite3.connect('sentiment_analysis.db')
df.to_sql('sentiments', conn, if_exists='replace', index=False)
conn.close()


In [14]:
import os

file_path = 'cleaned_sentiment_dataset.csv'

if os.path.isfile(file_path):
    print(f"The file '{file_path}' has been created successfully.")
else:
    print(f"The file '{file_path}' does not exist.")


The file 'cleaned_sentiment_dataset.csv' has been created successfully.


In [15]:
import os

directory = '.'

# List all files in the directory
files = os.listdir(directory)

if file_path in files:
    print(f"The file '{file_path}' is in the directory.")
else:
    print(f"The file '{file_path}' is not found in the directory.")


The file 'cleaned_sentiment_dataset.csv' is in the directory.


In [16]:
import pandas as pd

try:
    df_check = pd.read_csv(file_path)
    print(f"The file '{file_path}' has been read successfully.")
    print(df_check.head())
except FileNotFoundError:
    print(f"The file '{file_path}' does not exist.")


The file 'cleaned_sentiment_dataset.csv' has been read successfully.
   Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   

                                                Text    Sentiment  \
0   Enjoying a beautiful day at the park!        ...   Positive     
1   Traffic was terrible this morning.           ...   Negative     
2   Just finished an amazing workout! 💪          ...   Positive     
3   Excited about the upcoming weekend getaway!  ...   Positive     
4   Trying out a new recipe for dinner tonight.  ...   Neutral      

             Timestamp            User     Platform  \
0  2023-01-15 12:30:00   User123          Twitter     
1  2023-01-15 08:45:00   CommuterX        Twitter     
2  2023-01-15 15:45:00   FitnessFan      Instagram    
3  2023-01-15 18:20:00   AdventureX       Facebook    
4  2023-01-15 19:55:00   ChefCook        Instagram    

In [17]:
df.to_csv('sentimentdataset.csv')